In [10]:
import json
import sys
import collections
import copy

In [2]:
# import re
# import pandas as pd

# ans_pattern = re.compile(r'@content\d@')
# def is_multi_answer(answer):
#     docs = ans_pattern.findall(answer)
#     return len(set([int(doc[-2:-1]) for doc in docs])) > 1

# train_df = pd.read_csv('../input/original/train_round_0.csv')
# train_df['is_multi_answer'] = train_df['answer'].map(is_multi_answer)

In [3]:
# from eval_metric import normalize, compute_bleu_rouge

## 规则方法得到多答案问题id

In [5]:
question_dict = {}
with open('/home/lq/projects/Research/Reading-Comprehension/les-military-mrc/input/answer_mrc_dataset/test_r0.json') as f:
    for line in f:
        sample = json.loads(line)
        question_dict[sample['question_id']] = sample['question']

In [6]:
len(question_dict)

4969

In [7]:
cnt = 0
multi_ans_question_id_list = []
for id_, question in question_dict.items():
    if (question.count('?') + question.count('？')) > 1 or (question.count('多少') + question.count('哪') + question.count('什么') + question.count('谁')) > 1:
        cnt += 1
        multi_ans_question_id_list.append(id_)
#     elif '分别' in question and '和' in question:
#         print(id_, question)
#     elif '哪两' in question:
#         print(id_, question)
    
print(cnt)

285


In [8]:
multi_ans_question_id_list += ['Q_06031cjhhh', 'Q_47760knsdc', 'Q_52500cnjsq', 'Q_36961chjcs', 'Q_22570hycqj',
                               'Q_40246dnjsn']

In [9]:
len(multi_ans_question_id_list)

291

## bert classifier模型得到多答案id

In [11]:
is_multi_list = []
with open('pred_results.txt') as f:
    for line in f:
        is_multi_list.append(int(line.strip()))
print(len(is_multi_list))

4969


In [12]:
multi_ans_new = []
with open('/home/lq/projects/Research/Reading-Comprehension/les-military-mrc/input/answer_mrc_dataset/test_r0.json') as f:
    for i, line in enumerate(f):
        sample = json.loads(line)
        if is_multi_list[i] == 1:
            multi_ans_new.append(sample['question_id'])
#             if sample['question_id'] not in multi_ans_question_id_list:
#                 print(sample['question'])
#                 pass
print(len(multi_ans_new))

300


In [13]:
multi_ans_new += ['Q_06031cjhhh', 'Q_47760knsdc', 'Q_52500cnjsq', 'Q_36961chjcs', 'Q_22570hycqj',
                               'Q_40246dnjsn']
multi_ans_new = list(set(multi_ans_new))
print(len(multi_ans_new))

303


## 生成多答案

In [14]:
with open('../les_mrc/span_mrc/answer_models/answer_mrc_BertForLes_no-bridge-entity_back-trans_0928/checkpoint-best/nbest_predictions_checkpoint_test.json') as f:
    all_nbest_json = json.loads(f.read())

In [15]:
all_samples = collections.defaultdict(list)
need_skippeed_list = [',', '.', '。']
for qas_id, nbest_json in all_nbest_json.items():
    text = ''
    prob = 0.0
    logit = 0.0
    for entry in nbest_json:
        if entry['text'].strip() and entry['text'] not in need_skippeed_list:
            text = entry['text'].strip()
            logit = entry['start_logit'] + entry['end_logit']
            prob = entry['probability']
            break
    all_samples[qas_id.split('##')[0]].append([text, logit, prob])
all_predictions = {}
multi_pred = []
cnt = 0
for question_id, sample in all_samples.items():
    sample = sorted(sample, key=lambda x: x[1], reverse=True)
    all_predictions[question_id] = sample[0][0]
    # 简单的多答案选择模块
    if question_id in multi_ans_new:
#         for i in range(1, 5):
#             ans1 = normalize([sample[0][0]])
#             ans2 = normalize([sample[i][0]])
#         if sample[0][0] == sample[1][0]:
#             continue
        all_predictions[question_id] = all_predictions[question_id] + '#' + sample[1][0]
        print(all_predictions[question_id])
        cnt += 1
#     if sample[1][0] != '' and sample[1][2] > 0.5:
#         # 有可能具有多答案
#         if sample[0][0] == sample[1][0]:
#             continue
#         if sample[1][0] in sample[0][0] or sample[0][0] in sample[1][0]:
#             continue
#         ans1 = normalize([sample[0][0]])
#         ans2 = normalize([sample[1][0]])
#         bleu_rouge = compute_bleu_rouge({'item': ans1}, {'item': ans2})
#         if bleu_rouge['Bleu-4'] > 0.2 or bleu_rouge['Rouge-L'] > 0.2:
#             continue
#         print('{} have multi-ans, take care of it'.format(question_id))
#         multi_pred.append(question_id)
#         cnt += 1
#         all_predictions[question_id] = all_predictions[question_id] + '#' + sample[1][0]
        
print(cnt)

货舱宽度为5.49米#货舱宽度为5.49米
M16突击步枪#有效射程：885米
在F-104战斗机机腹下方安装了一台固体火箭发动机。#1951年开始研制
整车高宽比为0.91#5吨
英国、西班牙、德国和瑞士#棉兰老岛正成为“伊斯兰国”在东南亚
两具#12艘
可能会配备现在正在开发的远程防区外武器(LRSO)#飞翼结构
最大的特点是射速快，后座力低#11.43毫米
瑞典萨博(SAAB)公司#美国波音公司
长约8米#由南非桑达克·奥斯特瑞尔·比派克公司
气动、气冷原理#采用气动、气冷原理
公路机动模式#卡车拖挂式运载
突破了大型舰艇总体设计、信息集成、总装建造等一系列关键技术#中国船舶重工集团701研究所设计
25辆#“豹”1主战坦克
MG42#800米以上
15.6公里#最大射程18公里
MIM-23“霍克”地空导弹#184公里
西班牙内战#30年代
全新的航空复合材料#8吨
40-50米水平#东风11导弹的技术
比利时小镇巴斯通#OH-58D型“基奥瓦”侦察直升机
20-40千米之间#AA-11射手(Archer)
效费比#小型喷气发动机提供动力，飞行速度只有0、8马赫。
240艘#诺斯罗普格鲁门公司生产的“传奇”级（Legend）国家安全舰
发动机马力提升了190#誉21型发动机
布雷默顿预备役舰队#42架
SS-X-29#其发射车重量为80吨
枪身长度为1138毫米#射击稳定性好，攻击力强
沱江级舰具匿踪能力，速度快特性，并能够携带8枚雄风二型与8枚雄风三型反舰导弹，火力强大。#舰上搭载16枚杀伤力强大的雄风系列反舰导弹。
12门280毫米45倍径主炮#280毫米口径主炮
采用了两根枪管#3.31千克
123架#3架美制C-130运输机
约为四千吨左右#1000吨
莫斯科“劳动旗帜”工厂（现米格公司第二生产中心）#63吨
一个中国原则和中美三个联合公报#一个中国原则和中美三个联合公报
1984-1991年#"特拉法加"级攻击型核潜艇
飞行速度为1.78马赫#高强度、耐高温的BMI复合材料
飞行速度达到了8马赫，约9800公里每小时#5马赫
B-2隐身战略轰炸机#61米
20毫米#共计交付了1541架
2架#巴基斯坦从中国采购了一批霹雳15远程空空导弹
25发一小节的分段式结构#可歼灭1000米以内的集团目标
134米#134米
全枪重量为3.45千克#14颗
1

In [16]:
with open('add_back_trans_combine_0315pred-entity_0928.json', "w") as writer:
    writer.write(json.dumps(all_predictions, indent=4, ensure_ascii=False) + "\n")